# Sowing Success: How Machine Learning Helps Farmers Select the Best Crops


<img src="https://cdn11.bigcommerce.com/s-jl3t5tg/product_images/uploaded_images/rwanda7.jpg" width="800" height="200"/>

Measuring essential soil metrics such as nitrogen, phosphorous, potassium levels, and pH value is an important aspect of assessing soil condition. However, it can be an expensive and time-consuming process, which can cause farmers to prioritize which metrics to measure based on their budget constraints.

Farmers have various options when it comes to deciding which crop to plant each season. Their primary objective is to maximize the yield of their crops, taking into account different factors. One crucial factor that affects crop growth is the condition of the soil in the field, which can be assessed by measuring basic elements such as nitrogen and potassium levels. Each crop has an ideal soil condition that ensures optimal growth and maximum yield.

A farmer reached out to you as a machine learning expert for assistance in selecting the best crop for his field. They've provided you with a dataset called soil_measures.csv, which contains:

- "N": Nitrogen content ratio in the soil
- "P": Phosphorous content ratio in the soil
- "K": Potassium content ratio in the soil
- "pH" value of the soil
- "crop": categorical values that contain various crops (target variable).
Each row in this dataset represents various measures of the soil in a particular field. Based on these measurements, the crop specified in the "crop" column is the optimal choice for that field.

In this project, we will build multi-class classification models to predict the type of "crop" and identify the single most importance feature for predictive performance.

In [1]:
import pandas as pd
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import LabelEncoder
from sklearn import metrics
import numpy as np
import shap
import matplotlib.pyplot as plt

In [2]:
crops = pd.read_csv("soil_measures.csv")

FileNotFoundError: [Errno 2] No such file or directory: 'soil_measures.csv'

In [ ]:
crops.head()

In [ ]:
features = ["N", "P", "K", "ph"]

In [ ]:
X = crops[features]
y = crops["crop"]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
scaler = StandardScaler()

X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [ ]:
le = LabelEncoder()

y_train = le.fit_transform(y_train)
y_test = le.transform(y_test)

In [ ]:
X_train = pd.DataFrame(X_train, columns=features)
X_test = pd.DataFrame(X_test, columns=features)

In [ ]:
log_reg = LogisticRegression(solver="lbfgs")
log_reg.fit(X_train, y_train)

## Feature Coefficients

In [ ]:
log_reg.coef_

In [ ]:
coefficient_means = np.mean(abs(log_reg.coef_), axis=0)
for feature, mean in zip(features, coefficient_means):
    print(f"The mean coefficient of the feature {feature} is {mean:.2f}")

We now have the relative importance, assuming linear relationships. However, this doesn't measure how prediction performance would change if a feature was removed.

## Model Performance when trained on each feature: F1 score

In [ ]:
performance = {}

for feature in features:
    log = LogisticRegression(solver="lbfgs", max_iter=300)
    log.fit(X_train[[feature]], y_train)
    y_pred = log.predict(X_test[[feature]])

    f1 = metrics.f1_score(y_test, y_pred, average="macro")
    performance[feature] = f1

performance

It is clear that Potassium, "K" has the highest raw predictive power for determining optimal crop choice for a field. Nitrogen, "N" follows closely, while the PH balance of the soil looks almost insiginificant. However, we need to take into accound the relationships between the feaures themselves. Maybe PH + P exudes more influence than N alone. And that is why we are going to explore SHAP. SHAP (SHapley Additive exPlanations) is one of the most powerful and interpretable tools to understand feature importance and individual predictions.  

## SHAP 
"The core idea behind ShAP value based explanations of machine learning models is to use fair allocation results from cooperative game theory to allocate credit for a model’s output 
among its input features. In order to connect game theory with machine learning models, it is necessary to both match a model’s input features with players in a game, and also match the model function with the rules of the game. Since in game theory a player can join or not join a game, we need a way for a feature to “join” or “not join” a model." — Official docs
https://shap.readthedocs.io/en/stable/example_notebooks/overviews/An%20introduction%20to%20explainable%20AI%20with%20Shapley%20values.html

In [ ]:
shap.initjs()

In [ ]:
explainer = shap.LinearExplainer(log_reg, X_train)
shap_values = explainer(X_train)

### Individual Data Point Feature Contribution: Feature-wise contribution in the model's classification of a particular data point into a particular class. 
Waterfall plots are designed to display explanations for individual predictions. The bottom of a waterfall plot starts as the expected value of the model output, and then each row shows how the positive (red) or negative (blue) contribution of each feature moves the value from the expected model output over the background dataset to the model output for this prediction.

In this case, we are going to be choosing a data point at random, then explore each feature's individual contribution to the model's decision to classify it into a particular crop class.

In [ ]:
num_classes = shap_values.values.shape[2]
n = len(X_train)

In [ ]:
for _ in range(4):
    index = np.random.randint(n)  # Random data point
    class_index = np.random.randint(num_classes)  # Random class index
    class_name = le.inverse_transform([class_index])[0]  # Crop name from earlier LabelEncoder
    
    explanation = shap.Explanation(
        values=shap_values[:, :, class_index],
        base_values=shap_values.base_values[:, class_index],
        data=X_train,
        feature_names=features
    )

    shap.plots.waterfall(explanation[index], show=False)
    plt.title(f"Sample {index}, Class: {class_name}")
    plt.show()


### Class-wise Feature Contribution: Feature-wise contribution in the model's classification of all points to a particular class. 
Now, let’s explore the model’s behavior crop-wise. For each of the five randomly selected crop classes out of our total of twenty-two, we will examine how the individual features influence the model’s decision to classify all data points that were classified into that specific crop class.

In [ ]:
num_classes = len(le.classes_)

for _ in range(5):
    class_index = np.random.randint(num_classes)
    class_name = le.classes_[class_index]
    print(f"SHAP summary plot for class: {class_name}")
    shap.summary_plot(
        shap_values[:, :, class_index],  
        X_train,                         # feature data
        feature_names=features,
        show=True
    )

## Dependence Scatter Plot

A dependence scatter plot illustrates how a single feature influences the model's predictions.
In this case, the plot uses SHAP values aggregated across all 22 crop classes.

To highlight potential interaction effects between features, the `color=explanation` argument is used. This reveals how the selected feature interacts with others—specifically, the one it most frequently pairs with in driving the prediction is shown on the right side of the graph.

In [ ]:
for feature in features:
    shap.plots.scatter(explanation[:, feature], color=explanation)

## The bigger picture: How each feature influences classification accross all classes.

In [ ]:
# Aggregate SHAP values across classes
shap_means = np.mean(np.abs(shap_values.values), axis=2)  

#Use mean base values 
base_vals = np.mean(shap_values.base_values, axis=1) 

exp = shap.Explanation(
    values=shap_means,
    base_values=base_vals,
    data=X_train,
    feature_names=features
)

shap.plots.bar(exp)